In [1]:
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import requests
import os
import pandas as pd
import numpy as np
import folium

In [2]:
os.chdir("C:\A-Personal\AI2\CourseaProjects\DA-1\Class3-Week3")

In [3]:
# httpResponseCotent = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
httpResponseCotent = None
with open("ListCM.htm",encoding='UTF-8') as html_file:
    httpResponseCotent = BeautifulSoup(html_file,"lxml")

In [4]:
# data transform from HTML inner text to data frame
# get target table
data = httpResponseCotent.find("table",class_="wikitable sortable")
# get targe tr list
dataRows = data.findAll("tr")
# dictionary as data container
d = {'Postcode':[],'Borough':[],'Neighbourhood':[]}
# to loop the tr list and put the value to the dictionary

for index in range(1,len(dataRows)):
#   get values for the fiels
    Postcode = dataRows[index].findAll('td')[0].text
    Borough = dataRows[index].findAll('td')[1].text.replace('\n','')
    Neighbourhood = dataRows[index].findAll('td')[2].text.replace('\n','')
    
#   ignore the row with no Borough assigned
    if(Borough == 'Not assigned'):
        continue
#   assign same value from borough to neighbourhodd if it's not assigned 
    if(Neighbourhood == 'Not assigned'):
        Neighbourhood = Borough
    
    if(d['Postcode'].count(Postcode)>0):
        postCodeIndex = d['Postcode'].index(Postcode)
        d['Neighbourhood'][postCodeIndex] = d['Neighbourhood'][postCodeIndex] + ',' + Borough
        continue
    
    d['Postcode'].append(Postcode)
    d['Borough'].append(Borough)
    d['Neighbourhood'].append(Neighbourhood)
    
# convert the dictionary to data frame
df_Mlist = pd.DataFrame(data=d)
print('The shape of the data frame is {}'.format(df_Mlist.shape))
df_Mlist.head()

The shape of the data frame is (103, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Downtown Toronto"
3,M6A,North York,"Lawrence Heights,North York"
4,M7A,Queen's Park,Queen's Park


In [5]:
# get the latitue and longitue information from CSV
df_latlng = pd.read_csv("Geospatial_Coordinates.csv")
df_latlng = df_latlng.rename(index=str, columns={"Postal Code":"Postcode"})
print('The shape of the data frame is {}'.format(df_latlng.shape))
df_latlng.head()

The shape of the data frame is (103, 3)


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
# inner join the data between df_Mlist and df_latlng by Postcode
df_final = pd.merge(df_Mlist,df_latlng)
print('The shape of the data frame is {}'.format(df_final.shape))
df_final.head()

The shape of the data frame is (103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Downtown Toronto",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,North York",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [7]:
df_Toronto = df_final[df_final["Borough"].str.contains("Toronto")]
print('The shape of the data frame is {}'.format(df_Toronto.shape))
df_Toronto.head()

The shape of the data frame is (38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Downtown Toronto",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Downtown Toronto",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [8]:
# get the geo information about North York
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [9]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11.5)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [10]:
df_Toronto[0:1]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Downtown Toronto",43.65426,-79.360636


In [11]:
import json

# init the information about the certification
CLIENT_ID = 'A3EBICYRNY3NN5DCUIPZTUPWHRMM5EWDZPXOZW4PIDZH0BKD' 
CLIENT_SECRET = 'MA3U0L22R0YELKNWSTDRGQ3ZXJQRPJKZKXSBX0ZVHBIRUOA1' 
VERSION = '20190618' 
LIMIT = 100
radius = 100
neighborhood_latitude = '43.65426'
neighborhood_longitude = '-79.360636'

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('The URL:' + url)

Your credentails:
CLIENT_ID: A3EBICYRNY3NN5DCUIPZTUPWHRMM5EWDZPXOZW4PIDZH0BKD
CLIENT_SECRET:MA3U0L22R0YELKNWSTDRGQ3ZXJQRPJKZKXSBX0ZVHBIRUOA1
The URL:https://api.foursquare.com/v2/venues/explore?&client_id=A3EBICYRNY3NN5DCUIPZTUPWHRMM5EWDZPXOZW4PIDZH0BKD&client_secret=MA3U0L22R0YELKNWSTDRGQ3ZXJQRPJKZKXSBX0ZVHBIRUOA1&v=20190618&ll=43.65426,-79.360636&radius=100&limit=100


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
resultInfo = getNearbyVenues(df_Toronto["Neighbourhood"],df_Toronto["Latitude"],df_Toronto["Longitude"])

Harbourfront,Downtown Toronto
Ryerson,Downtown Toronto
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,Downtown Toronto,Downtown Toronto
Dovercourt Village,West Toronto
Harbourfront East,Downtown Toronto,Downtown Toronto
Little Portugal,West Toronto
The Danforth West,East Toronto
Design Exchange,Downtown Toronto
Brockton,West Toronto,West Toronto
The Beaches West,East Toronto
Commerce Court,Downtown Toronto
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Central Toronto
High Park,West Toronto
North Toronto West
The Annex,Central Toronto,Central Toronto
Parkdale,West Toronto
Davisville
Harbord,Downtown Toronto
Runnymede,West Toronto
Moore Park,Central Toronto
Chinatown,Downtown Toronto,Downtown Toronto
Deer Park,Central Toronto,Central Toronto,Central Toronto,Central Toronto
CN Tower,Downtown Toronto,Downtown Toronto,Downtown Toronto,Downtown Toronto,Downtown Toronto,Downtown Toronto
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbageto

In [14]:
print(resultInfo.shape)
resultInfo.head()

(1700, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Downtown Toronto",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Downtown Toronto",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Downtown Toronto",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Downtown Toronto",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront,Downtown Toronto",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [15]:
# to check the venue category information
resultInfo.groupby(["Venue Category"]).count().sort_values(by='Venue Longitude',ascending=False).head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Coffee Shop,151,151,151,151,151,151
Café,91,91,91,91,91,91
Restaurant,54,54,54,54,54,54
Italian Restaurant,48,48,48,48,48,48
Bakery,45,45,45,45,45,45


In [16]:
# one hot encoding
resultInfo_onehot = pd.get_dummies(resultInfo[['Venue Category']], prefix="", prefix_sep="")
resultInfo_onehot.pop("Neighborhood")
resultInfo_onehot['Neighborhood'] = resultInfo['Neighborhood'] 
resultInfo_onehot.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Harbourfront,Downtown Toronto"
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Harbourfront,Downtown Toronto"
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Harbourfront,Downtown Toronto"
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Harbourfront,Downtown Toronto"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Harbourfront,Downtown Toronto"


In [118]:
resultInfo_grouped = resultInfo_onehot.groupby('Neighborhood').std().reset_index()
resultInfo_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,Downtown Toronto,Downtown Toronto",0.0,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.196946,...,0.0,0.0,0.10000,0.0,0.0,0.0,0.1,0.0,0.0,0.000000
1,Berczy Park,0.0,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.13484,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,"Brockton,West Toronto,West Toronto",0.0,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.213201
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.229416
4,"CN Tower,Downtown Toronto,Downtown Toronto,Dow...",0.0,0.0,0.25,0.25,0.25,0.341565,0.341565,0.341565,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [121]:
num_top_venues = 5

for hood in resultInfo_grouped['Neighborhood']:
    print("----"+str(hood)+"----")
    temp = resultInfo_grouped[resultInfo_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,Downtown Toronto,Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.24
1                 Café  0.22
2                  Bar  0.20
3           Steakhouse  0.20
4  American Restaurant  0.20


----Berczy Park----
                venue  freq
0         Coffee Shop  0.29
1        Cocktail Bar  0.23
2              Bakery  0.19
3         Cheese Shop  0.19
4  Seafood Restaurant  0.19


----Brockton,West Toronto,West Toronto----
            venue  freq
0            Café  0.29
1  Breakfast Spot  0.29
2     Coffee Shop  0.29
3     Yoga Studio  0.21
4             Gym  0.21


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0    Yoga Studio  0.23
1     Restaurant  0.23
2            Spa  0.23
3  Burrito Place  0.23
4        Butcher  0.23


----CN Tower,Downtown Toronto,Downtown Toronto,Downtown Toronto,Downtown Toronto,Downtown Toronto,Downtown Toronto----
              venue  freq
0    Airport Lounge  0.34
1   Airport Service  0.34

In [154]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = resultInfo_grouped['Neighborhood']

for ind in np.arange(resultInfo_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(resultInfo_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,Downtown Toronto,Downtown Toronto",Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Hotel,Cosmetics Shop,Bakery,Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Italian Restaurant,Beer Bar,Steakhouse,Cheese Shop,Farmers Market,Café
2,"Brockton,West Toronto,West Toronto",Coffee Shop,Café,Breakfast Spot,Yoga Studio,Convenience Store,Italian Restaurant,Climbing Gym,Restaurant,Caribbean Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Spa,Farmers Market,Park,Comic Shop,Pizza Place,Recording Studio,Butcher,Burrito Place,Brewery
4,"CN Tower,Downtown Toronto,Downtown Toronto,Dow...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Airport,Airport Food Court,Boat or Ferry,Harbor / Marina,Coffee Shop,Boutique


In [155]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

resultInfo_grouped_clustering = resultInfo_grouped.drop('Neighborhood', 1)
resultInfo_grouped_clustering.head()

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(resultInfo_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 3, 0, 4, 1, 4, 4, 4, 0, 4, 3, 4, 0, 4, 3, 1, 3, 2, 4, 3, 3, 0,
       2, 1, 0, 4, 1, 2, 1, 4, 4, 3, 3, 4, 4, 2, 4, 4])

In [156]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

resultInfo_merged = df_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
resultInfo_merged = resultInfo_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

resultInfo_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront,Downtown Toronto",43.654260,-79.360636,3,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Theater,Restaurant,Beer Store
9,M5B,Downtown Toronto,"Ryerson,Downtown Toronto",43.657162,-79.378937,4,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Ramen Restaurant,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Theater
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Café,Coffee Shop,Hotel,Restaurant,Bakery,Cosmetics Shop,Cocktail Bar,Clothing Store,Breakfast Spot,Gastropub
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Trail,Pub,Other Great Outdoors,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Italian Restaurant,Beer Bar,Steakhouse,Cheese Shop,Farmers Market,Café


In [157]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(resultInfo_merged['Latitude'], resultInfo_merged['Longitude'], resultInfo_merged['Neighbourhood'], resultInfo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters